### The data “gdp.txt” consists of U.S. quarterly real GDP from 1947 to the 2nd quarter of 2005. The data are seasonally adjusted. [There are four columns in the file; year, month, day and GDP.] Build an adequate model for the data, including model checking and outlier detection. Produce 1- to 5-step ahead forecasts at the forecast origin July 2005 (i.e. the last data point).


Step1: First we draw the time series plot. We can investigate the data whether it has any pattern or unusual observation. This time-series data, named gdp, does not have an outlier deviating from a time series plot. However, gdp has a trend. This is obvious that the data is not stationary, and we might need transformation or differencing the data.   

will be added shortly.